# Mine

In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.lookup_tools import extractor
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import crystalball
import matplotlib.pyplot as plt
from hist import Hist
from coffea.analysis_tools import PackedSelection
from scipy.integrate import quad

from coffea.ml_tools.torch_wrapper import torch_wrapper
from analyzer.core import analyzerModule
from analyzer.matching import object_matching
import pickle

import torch
import torch.nn as nn
import torch.jit as jit
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import pandas as pd

In [2]:
class jetAssignmentNN(torch_wrapper):
    def prepare_awkward(self,events):
        ak = self.get_awkward_lib(events)
        jets = events.Jet.good_jets
        flat_jets = ak.flatten(jets)

        m3 = jets[:,1:4].sum()
        m4 = jets[:,0:4].sum()

        ones = ak.ones_like(jets.pt)

        imap = {
            "features": {
                "jetOrdinality":    ak.flatten(ak.local_index(jets, axis=1)),
                "jetPT": 		    flat_jets.pt,
                "jetEta": 		    flat_jets.eta,
                "jetPhi": 		    flat_jets.phi,
                "jetBScore":    	flat_jets.btagDeepFlavB,
                "m3M": 			    ak.flatten(ones * m3.mass),
                "m3PT": 		    ak.flatten(ones * m3.pt),
                "m3Eta": 		    ak.flatten(ones * m3.eta),
                "m3Phi": 	        ak.flatten(ones * m3.phi),
                "m4M": 			    ak.flatten(ones * m4.mass),
                "m4PT":		        ak.flatten(ones * m4.pt),
                "m4Eta":		    ak.flatten(ones * m4.eta),
                "m4Phi":	        ak.flatten(ones * m4.phi)
            }
        }

        retmap = {
            "features": ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()])
        }

        return(),{
            "features": ak.values_astype(retmap["features"],"float32")
        }

In [12]:
torch.rand(13)

tensor([0.8474, 0.1920, 0.4003, 0.5021, 0.4244, 0.4930, 0.7847, 0.4848, 0.8324,
        0.0420, 0.6339, 0.0729, 0.0080])

In [18]:
traced_module = torch.jit.trace(model, torch.rand(13))

/usr/local/lib/python3.10/site-packages/torch/jit/_trace.py:787: UserWarning: The input to trace is already a ScriptModule, tracing it is a no-op. Returning the object as is.
  warnings.warn(


In [8]:
model = torch.load("jetMatcherNN.pt")

In [3]:
model_new = jetAssignmentNN("jetMatcherNN.pt")

In [11]:
model_new(events)

/usr/local/lib/python3.10/site-packages/coffea/ml_tools/helper.py:175: UserWarning: No format checks were performed on input!
  warnings.warn("No format checks were performed on input!")


RuntimeError: PytorchStreamReader failed locating file constants.pkl: file not found

In [4]:
class Net(jit.ScriptModule):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(13,13)
        self.fc2 = nn.Linear(13,3)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x),dim=1)
        return x

In [5]:
#TODO: fix path issue
fname = "data/NANOAOD-1500_900.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DYJets"},
    delayed=False
).events()

/usr/local/lib/python3.10/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [6]:
def makeCutSet(x, s, *args):
    return [x[s > a] for a in args]

b_tag_wps = [0.0490, 0.2783, 0.7100]

def createObjects(events):
    good_jets = events.Jet[(events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.4)]
    fat_jets = events.FatJet[(events.FatJet.pt > 30) & (abs(events.FatJet.eta) < 2.4)]
    loose_top, med_top, tight_top = makeCutSet(
        fat_jets, fat_jets.particleNet_TvsQCD, 0.58, 0.80, 0.97
    )
    #loose_W, med_W, tight_W = makeCutSet(
    #    fat_jets, fat_jets.particleNet_WvsQCD, 0.7, 0.94, 0.98
    #)

    #deep_top_wp1, deep_top_wp2, deep_top_wp3, deep_top_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_TvsQCD, 0.436, 0.802, 0.922, 0.989
    #)
    #deep_W_wp1, deep_W_wp2, deep_W_wp3, deep_W_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_WvsQCD, 0.458, 0.762, 0.918, 0.961
    #)
    loose_b, med_b, tight_b = makeCutSet(
        good_jets, good_jets.btagDeepFlavB, *(b_tag_wps[x] for x in range(3))
    )

    el = events.Electron
    good_electrons = el[
        (el.cutBased == 4)
        & (el.miniPFRelIso_all < 0.1)
        & (el.pt > 30)
        & (abs(el.eta) < 2.4)
    ]
    mu = events.Muon
    good_muons = mu[
        (mu.mediumId) & (mu.miniPFRelIso_all < 0.2) & (mu.pt > 30) & (abs(mu.eta) < 2.4)
    ]
    events["good_jets"] = good_jets
    events["good_electrons"] = good_electrons
    events["good_muons"] = good_muons

    events["loose_bs"] = loose_b
    events["med_bs"] = med_b
    events["tight_bs"] = tight_b

    events["tight_tops"] = tight_top
    # events["med_tops"] = med_top
    # events["loose_tops"] = loose_top

    # events["tight_Ws"] = tight_W
    # events["med_Ws"] = med_W
    # events["loose_Ws"] = loose_W

    # events["deep_top_wp1"] = deep_top_wp1
    # events["deep_top_wp2"] = deep_top_wp2
    # events["deep_top_wp3"] = deep_top_wp3
    # events["deep_top_wp4"] = deep_top_wp4

    # events["deep_W_wp1"] = deep_W_wp1
    # events["deep_W_wp2"] = deep_W_wp2
    # events["deep_W_wp3"] = deep_W_wp3
    # events["deep_W_wp4"] = deep_W_wp4

    return events

def createSelection(events):
    """Baseline selection for the analysis.
Applies the following selection:
- Jets[0].pt > 300
- 4 <= nJets <= 6
- 0 leptons
- 2 medium bjets, at least one of which is tight
- delta_R(med_bjets[0],med_bjets[1]) > 1
    """

    selection = PackedSelection()
    good_jets = events.good_jets
    fat_jets = events.FatJet
    good_muons = events.good_muons
    good_electrons = events.good_electrons
    loose_b = events.loose_bs
    med_b = events.med_bs
    tight_b = events.tight_bs
    tight_top = events.tight_tops
    # selection = PackedSelection()
    filled_jets = ak.pad_none(good_jets, 4, axis=1)
    top_two_dr = ak.fill_none(filled_jets[:, 0].delta_r(filled_jets[:, 1]), False)

    filled_med = ak.pad_none(med_b, 2, axis=1)
    med_dr = ak.fill_none(filled_med[:, 0].delta_r(filled_med[:, 1]), False)
    # selection.add("trigger", (ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6))
    if "HLT" in events.fields:
        selection.add("hlt", (events.HLT.PFHT1050 | events.HLT.AK8PFJet360_TrimMass30))
    selection.add("highptjet", (ak.fill_none(filled_jets[:, 0].pt > 300, False)))
    selection.add("jets", ((ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6)))
    selection.add("0Lep", ((ak.num(good_electrons) == 0) & (ak.num(good_muons) == 0)))
    selection.add("2bjet", (ak.num(med_b) >= 2))
    selection.add("1tightbjet", (ak.num(tight_b) >= 1))
    # selection.add("jet_dr", ((top_two_dr < 4) & (top_two_dr > 2)).to_numpy())
    selection.add("b_dr", (med_dr > 1))
    return selection

In [7]:
events = createObjects(events)
mask = createSelection(events).all("highptjet", "jets", "0Lep", "2bjet", "1tightbjet", "b_dr")
events = events[mask]

In [18]:
ak.zip([x[:, np.newaxis] for x in imap['features'].values()])

NameError: name 'imap' is not defined

In [12]:
ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()], axis=1).to_numpy()

NameError: name 'imap' is not defined

In [ ]:
class Net(nn.Module):
        def __init__(self):
            super(Net,self).__init__()
            self.fc1 = nn.Linear(13,13)
            self.fc2 = nn.Linear(13,3)
        def forward(self,x):
            x = F.relu(self.fc1(x))
            x = F.softmax(self.fc2(x),dim=1)
            return x

In [19]:
jets = events.good_jets
flat_jets = ak.flatten(jets)

m3 = jets[:,1:4].sum()
m4 = jets[:,0:4].sum()

ones = ak.ones_like(jets.pt)

imap = {
    "features": {
        "jetOrdinality":    ak.flatten(ak.local_index(jets, axis=1)),
        "jetPT": 		flat_jets.pt,
        "jetEta": 		flat_jets.eta,
        "jetPhi": 		flat_jets.phi,
        "jetBScore": 	flat_jets.btagDeepFlavB,
        "m3M": 			ak.flatten(ones * m3.mass),
        "m3PT": 		ak.flatten(ones * m3.pt),
        "m3Eta": 		ak.flatten(ones * m3.eta),
        "m3Phi": 		ak.flatten(ones * m3.phi),
        "m4M": 			ak.flatten(ones * m4.mass),
        "m4PT":			ak.flatten(ones * m4.pt),
        "m4Eta":		ak.flatten(ones * m4.eta),
        "m4Phi":	    ak.flatten(ones * m4.phi)
    }
}

model = torch.load("jetMatcherNN.pt")

net_input = torch.Tensor(ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()], axis=1))
scl = open('scaler.pkl', 'rb')
scaler = pickle.load(scl)
net_input = scaler.transform(net_input)

outputs = model(torch.Tensor(net_input)).detach().numpy()

stop_probs = outputs[:,0]
charg_probs = outputs[:,1]
other_probs = outputs[:,2]

high_charg_score_mask = ak.unflatten(outputs[:,1] > 0.8, ak.num(jets))
top_3_charg_scores = ak.sort(ak.unflatten(outputs[:,1], ak.num(jets)), axis=1)[:, -3:]
top_3_idx = ak.argsort(ak.unflatten(outputs[:,1], ak.num(jets)), axis=1)[:, -3:]

top_3_charg_score_sum = jets[top_3_idx].sum()
m3_top_3_nn_charg_score = top_3_charg_score_sum.mass
m3_high_nn_charg_score = jets[high_charg_score_mask].sum().mass

/usr/local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


jetOrdinality,jetPT,jetEta,jetPhi,jetBScore,m3M,m3PT,m3Eta,m3Phi,m4M,m4PT,m4Eta,m4Phi,isStopMatched,isChiMatched,isOther

In [ ]:
class jetAssignmentNN(torch_wrapper):
    def prepare_awkward(self,events):
        awk = self.get_awkward_lib(events)
        
        jets = events.good_jets
        flat_jets = ak.flatten(jets)

        m3 = jets[:,1:4].sum()
        m4 = jets[:,0:4].sum()

        ones = ak.ones_like(jets.pt)

        imap = {
            "features": {
                "jetOrdinality":	ak.flatten(ak.local_index(jets, axis=1)),
                "jetPT": 		flat_jets.pt,
                "jetEta": 		flat_jets.eta,
                "jetPhi": 		flat_jets.phi,
                "jetBScore": 	flat_jets.btagDeepFlavB,
                "m3M": 			ak.flatten(ones * m3.mass),
                "m3PT": 		ak.flatten(ones * m3.pt),
                "m3Eta": 		ak.flatten(ones * m3.eta),
                "m3Phi": 		ak.flatten(ones * m3.phi),
                "m4M": 			ak.flatten(ones * m4.mass),
                "m4PT":			ak.flatten(ones * m4.pt),
                "m4Eta":		ak.flatten(ones * m4.eta),
                "m4Phi":	    ak.flatten(ones * m4.phi)
            }
        }

        scl = open('scaler.pkl', 'rb')
        scaler = pickle.load(scl)

        retmap = {
            "features": ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()], axis=1)
        }

        print(retmap)
        
        return(),{
            "features": awk.values_astype(retmap["features"],"float32")
        }

In [ ]:
jets = events.good_jets
flat_jets = ak.flatten(jets)

m3 = jets[:,1:4].sum()
m4 = jets[:,0:4].sum()

ones = ak.ones_like(jets.pt)
ones_flat = ak.ones_like(flat_jets.pt)

imap = {
    "features": {
        "jetOrdinality":	ak.flatten(ak.local_index(jets, axis=1)),
        "jetPT": 		flat_jets.pt,
        "jetEta": 		flat_jets.eta,
        "jetPhi": 		flat_jets.phi,
        "jetBScore": 	flat_jets.btagDeepFlavB,
        "m3M": 			ak.flatten(ones * m3.mass),
        "m3PT": 		ak.flatten(ones * m3.pt),
        "m3Eta": 		ak.flatten(ones * m3.eta),
        "m3Phi": 		ak.flatten(ones * m3.phi),
        "m4M": 			ak.flatten(ones * m4.mass),
        "m4PT":			ak.flatten(ones * m4.pt),
        "m4Eta":		ak.flatten(ones * m4.eta),
        "m4Phi":	    ak.flatten(ones * m4.phi)
    }
}

# this here!!

In [ ]:
ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()], axis=1)

<Array [[0, 448, 0.138, ..., 0.384, 2.03], ...] type='21803 * 13 * float64'>

In [7]:
def addEventLevelVars(events):
    ht = ak.sum(events.good_jets.pt, axis=1)
    events["HT"] = ht
    return events

In [8]:
def deltaRMatch(events, analyzer):
    matched_jets, matched_quarks, dr, idx_j, idx_q, _ = object_matching(
        events.good_jets, events.SignalQuarks, 0.2, 0.5, True
    )
    # print(f"IndexQ: {idx_q}")
    # print(f"IndexJ: {idx_j}")
    # print(f"MQ: {matched_quarks}")
    # print(f"MJ: {matched_jets}")
    # _, _, _, ridx_q, ridx_j, _ = object_matching(
    #    events.SignalQuarks, events.good_jets, 0.3, 0.5, True
    # )
    events["matched_quarks"] = matched_quarks
    events["matched_jets"] = matched_jets
    events["matched_dr"] = dr
    events["matched_jet_idx"] = idx_j
    return events, analyzer

In [9]:
def addNNScores(events):
    model = jetAssignmentNN("jetMatcherNN.pt")
    scores = model(events)
    print(scores)
    events["NNStopProb"]  = scores[:,0]
    events["NNChiProb"]   = scores[:,1]
    events["NNOtherProb"] = scores[:,2]
    return events

In [30]:
addNNScores(events)

{'features': array([[-1.3645788 ,  1.22131757,  0.15853927, ...,  0.15667853,
         0.16756647,  1.08975591],
       [-0.69924149,  0.65760067, -0.44302743, ...,  0.15667853,
         0.16756647,  1.08975591],
       [-0.03390419,  0.11125319,  0.54687597, ...,  0.15667853,
         0.16756647,  1.08975591],
       ...,
       [-0.69924149,  0.41600771, -1.80467194, ...,  0.17728131,
         0.61933107,  0.15841618],
       [-0.03390419, -0.33166723,  0.77753909, ...,  0.17728131,
         0.61933107,  0.15841618],
       [ 0.63143312, -0.60010385, -0.54576496, ...,  0.17728131,
         0.61933107,  0.15841618]])}


/usr/local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/site-packages/coffea/ml_tools/helper.py:175: UserWarning: No format checks were performed on input!
  warnings.warn("No format checks were performed on input!")


RuntimeError: PytorchStreamReader failed locating file constants.pkl: file not found

In [11]:
model = jetAssignmentNN("jetMatcherNN.pt")

In [5]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(13,13)
    self.fc2 = nn.Linear(13,3)
  def forward(self,x):
    x = F.relu(self.fc1(x))
    x = F.softmax(self.fc2(x),dim=1)
    return x

In [12]:
model = torch.load("jetMatcherNN.pt")

In [13]:
model_input = torch.Tensor(ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()], axis=1))

In [14]:
scl = open('scaler.pkl', 'rb')
scaler = pickle.load(scl)

/usr/local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
model_input = scaler.transform(model_input)

In [16]:
outputs = model(torch.Tensor(model_input))

In [27]:
outputs = outputs.detach().numpy()

In [29]:
normalized_outputs = np.round(outputs)

In [24]:
event_counts = ak.num(jets)

In [52]:
outputs[:,1] > 0.8

array([ True, False,  True,  True, False, False,  True, False,  True,
        True, False,  True,  True,  True, False, False, False,  True,
        True, False, False, False, False,  True,  True, False, False,
       False,  True,  True,  True, False, False,  True, False,  True,
        True, False,  True,  True,  True,  True,  True, False, False,
       False,  True,  True, False, False, False,  True,  True,  True,
       False, False, False, False,  True,  True, False,  True,  True,
        True, False,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True, False,  True,  True, False, False,
       False,  True,  True, False, False, False,  True,  True, False,
       False,  True,  True,  True, False, False, False,  True,  True,
       False,  True,  True,  True, False,  True, False,  True,  True,
       False,  True, False,  True,  True,  True, False,  True,  True,
        True, False, False,  True,  True, False, False,  True,  True,
       False, False,

In [39]:
ak.num(jets, axis=1)

<Array [6, 4, 5, 6, 6, 6, 5, 6, ..., 4, 4, 4, 5, 4, 5, 4] type='4469 * int64'>

In [54]:
ak.unflatten(outputs[:,1], event_counts)

<Array [[1, 0.0333, ..., 0.00702, 0.000154], ...] type='4469 * var * float32'>

In [80]:
top_3_idx = ak.argsort(ak.unflatten(outputs[:,1], event_counts), axis=1)[:, -3:]

In [89]:
ak.unflatten(outputs[:,1], event_counts)[top_3_idx]

<Array [[1, 1, 1], ..., [0.86, 0.985, 0.998]] type='4469 * var * float32'>

In [92]:
nn_charg_mask = ak.unflatten(outputs[:,1] > 0.8, ak.num(jets))
top_3_chargscores = ak.sort(ak.unflatten(outputs[:,1], ak.num(jets)), axis=1)[:, -3:]
top_3_idx = ak.argsort(ak.unflatten(outputs[:,1], ak.num(jets)), axis=1)[:, -3:]


In [98]:
top_3_charg_score_sum = jets[top_3_idx].sum()

In [100]:
top_3_charg_score_sum.mass

<Array [863, 834, 787, 507, ..., 937, 941, 680, 2.55e+03] type='4469 * float32'>

In [103]:
jets[nn_charg_mask].sum().mass

<Array [863, 834, 787, 330, ..., 438, 277, 2.55e+03] type='4469 * float32'>

In [97]:
top_3_chargscores

<Array [[1, 1, 1], ..., [0.86, 0.985, 0.998]] type='4469 * var * float32'>

In [96]:
top_3_charg_score_sum = top_3_chargscores.sum()
m3_top_3_nn_charg_score = top_3_charg_score_sum.mass()

AttributeError: no field named 'sum'

In [93]:
top_3_idx

<Array [[3, 0, 2], [3, 0, 2], ..., [...], [1, 3, 0]] type='4469 * var * int64'>

In [84]:
ak.any(ak.num(top_3_idx) != 3)

False

In [20]:
df = pd.read_csv('jets_1500_900.csv')
targets = df.values[:,-3:]

In [40]:
len(targets)

12854

In [32]:
ak.unflatten(ak.flatten(targets), event_counts)

ValueError: structure imposed by 'counts' does not fit in the array or partition at axis=0

This error occurred while calling

    ak.unflatten(
        <Array [0, 1, 0, 1, 0, 0, ..., 0, 1, 0, 0, 0, 1] type='38562 * float64'>
        <Array [6, 4, 5, 6, 6, 6, 5, ..., 4, 4, 5, 4, 5, 4] type='4469 * int64'>
    )

## ??

In [17]:
def makeCutSet(x, s, *args):
    return [x[s > a] for a in args]

b_tag_wps = [0.0490, 0.2783, 0.7100]

def createObjects(events):
    good_jets = events.Jet[(events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.4)]
    fat_jets = events.FatJet[(events.FatJet.pt > 30) & (abs(events.FatJet.eta) < 2.4)]
    loose_top, med_top, tight_top = makeCutSet(
        fat_jets, fat_jets.particleNet_TvsQCD, 0.58, 0.80, 0.97
    )
    #loose_W, med_W, tight_W = makeCutSet(
    #    fat_jets, fat_jets.particleNet_WvsQCD, 0.7, 0.94, 0.98
    #)

    #deep_top_wp1, deep_top_wp2, deep_top_wp3, deep_top_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_TvsQCD, 0.436, 0.802, 0.922, 0.989
    #)
    #deep_W_wp1, deep_W_wp2, deep_W_wp3, deep_W_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_WvsQCD, 0.458, 0.762, 0.918, 0.961
    #)
    loose_b, med_b, tight_b = makeCutSet(
        good_jets, good_jets.btagDeepFlavB, *(b_tag_wps[x] for x in range(3))
    )

    el = events.Electron
    good_electrons = el[
        (el.cutBased == 4)
        & (el.miniPFRelIso_all < 0.1)
        & (el.pt > 30)
        & (abs(el.eta) < 2.4)
    ]
    mu = events.Muon
    good_muons = mu[
        (mu.mediumId) & (mu.miniPFRelIso_all < 0.2) & (mu.pt > 30) & (abs(mu.eta) < 2.4)
    ]
    events["good_jets"] = good_jets
    events["good_electrons"] = good_electrons
    events["good_muons"] = good_muons

    events["loose_bs"] = loose_b
    events["med_bs"] = med_b
    events["tight_bs"] = tight_b

    events["tight_tops"] = tight_top
    # events["med_tops"] = med_top
    # events["loose_tops"] = loose_top

    # events["tight_Ws"] = tight_W
    # events["med_Ws"] = med_W
    # events["loose_Ws"] = loose_W

    # events["deep_top_wp1"] = deep_top_wp1
    # events["deep_top_wp2"] = deep_top_wp2
    # events["deep_top_wp3"] = deep_top_wp3
    # events["deep_top_wp4"] = deep_top_wp4

    # events["deep_W_wp1"] = deep_W_wp1
    # events["deep_W_wp2"] = deep_W_wp2
    # events["deep_W_wp3"] = deep_W_wp3
    # events["deep_W_wp4"] = deep_W_wp4

    return events

def createSelection(events):
    """Baseline selection for the analysis.
Applies the following selection:
- Jets[0].pt > 300
- 4 <= nJets <= 6
- 0 leptons
- 2 medium bjets, at least one of which is tight
- delta_R(med_bjets[0],med_bjets[1]) > 1
    """

    selection = PackedSelection()
    good_jets = events.good_jets
    fat_jets = events.FatJet
    good_muons = events.good_muons
    good_electrons = events.good_electrons
    loose_b = events.loose_bs
    med_b = events.med_bs
    tight_b = events.tight_bs
    tight_top = events.tight_tops
    # selection = PackedSelection()
    filled_jets = ak.pad_none(good_jets, 4, axis=1)
    top_two_dr = ak.fill_none(filled_jets[:, 0].delta_r(filled_jets[:, 1]), False)

    filled_med = ak.pad_none(med_b, 2, axis=1)
    med_dr = ak.fill_none(filled_med[:, 0].delta_r(filled_med[:, 1]), False)
    # selection.add("trigger", (ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6))
    if "HLT" in events.fields:
        selection.add("hlt", (events.HLT.PFHT1050 | events.HLT.AK8PFJet360_TrimMass30))
    selection.add("highptjet", (ak.fill_none(filled_jets[:, 0].pt > 300, False)))
    selection.add("jets", ((ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6)))
    selection.add("0Lep", ((ak.num(good_electrons) == 0) & (ak.num(good_muons) == 0)))
    selection.add("2bjet", (ak.num(med_b) >= 2))
    selection.add("1tightbjet", (ak.num(tight_b) >= 1))
    # selection.add("jet_dr", ((top_two_dr < 4) & (top_two_dr > 2)).to_numpy())
    selection.add("b_dr", (med_dr > 1))
    return selection

In [18]:
def makeCutSet(x, s, *args):
    return [x[s > a] for a in args]

b_tag_wps = [0.0490, 0.2783, 0.7100]

def createObjects(events):
    good_jets = events.Jet[(events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.4)]
    fat_jets = events.FatJet[(events.FatJet.pt > 30) & (abs(events.FatJet.eta) < 2.4)]
    loose_top, med_top, tight_top = makeCutSet(
        fat_jets, fat_jets.particleNet_TvsQCD, 0.58, 0.80, 0.97
    )
    #loose_W, med_W, tight_W = makeCutSet(
    #    fat_jets, fat_jets.particleNet_WvsQCD, 0.7, 0.94, 0.98
    #)

    #deep_top_wp1, deep_top_wp2, deep_top_wp3, deep_top_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_TvsQCD, 0.436, 0.802, 0.922, 0.989
    #)
    #deep_W_wp1, deep_W_wp2, deep_W_wp3, deep_W_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_WvsQCD, 0.458, 0.762, 0.918, 0.961
    #)
    loose_b, med_b, tight_b = makeCutSet(
        good_jets, good_jets.btagDeepFlavB, *(b_tag_wps[x] for x in range(3))
    )

    el = events.Electron
    good_electrons = el[
        (el.cutBased == 4)
        & (el.miniPFRelIso_all < 0.1)
        & (el.pt > 30)
        & (abs(el.eta) < 2.4)
    ]
    mu = events.Muon
    good_muons = mu[
        (mu.mediumId) & (mu.miniPFRelIso_all < 0.2) & (mu.pt > 30) & (abs(mu.eta) < 2.4)
    ]
    events["good_jets"] = good_jets
    events["good_electrons"] = good_electrons
    events["good_muons"] = good_muons

    events["loose_bs"] = loose_b
    events["med_bs"] = med_b
    events["tight_bs"] = tight_b

    events["tight_tops"] = tight_top
    # events["med_tops"] = med_top
    # events["loose_tops"] = loose_top

    # events["tight_Ws"] = tight_W
    # events["med_Ws"] = med_W
    # events["loose_Ws"] = loose_W

    # events["deep_top_wp1"] = deep_top_wp1
    # events["deep_top_wp2"] = deep_top_wp2
    # events["deep_top_wp3"] = deep_top_wp3
    # events["deep_top_wp4"] = deep_top_wp4

    # events["deep_W_wp1"] = deep_W_wp1
    # events["deep_W_wp2"] = deep_W_wp2
    # events["deep_W_wp3"] = deep_W_wp3
    # events["deep_W_wp4"] = deep_W_wp4

    return events

def createSelection(events):
    """Baseline selection for the analysis.
Applies the following selection:
- Jets[0].pt > 300
- 4 <= nJets <= 6
- 0 leptons
- 2 medium bjets, at least one of which is tight
- delta_R(med_bjets[0],med_bjets[1]) > 1
    """

    selection = PackedSelection()
    good_jets = events.good_jets
    fat_jets = events.FatJet
    good_muons = events.good_muons
    good_electrons = events.good_electrons
    loose_b = events.loose_bs
    med_b = events.med_bs
    tight_b = events.tight_bs
    tight_top = events.tight_tops
    # selection = PackedSelection()
    filled_jets = ak.pad_none(good_jets, 4, axis=1)
    top_two_dr = ak.fill_none(filled_jets[:, 0].delta_r(filled_jets[:, 1]), False)

    filled_med = ak.pad_none(med_b, 2, axis=1)
    med_dr = ak.fill_none(filled_med[:, 0].delta_r(filled_med[:, 1]), False)
    # selection.add("trigger", (ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6))
    if "HLT" in events.fields:
        selection.add("hlt", (events.HLT.PFHT1050 | events.HLT.AK8PFJet360_TrimMass30))
    selection.add("highptjet", (ak.fill_none(filled_jets[:, 0].pt > 300, False)))
    selection.add("jets", ((ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6)))
    selection.add("0Lep", ((ak.num(good_electrons) == 0) & (ak.num(good_muons) == 0)))
    selection.add("2bjet", (ak.num(med_b) >= 2))
    selection.add("1tightbjet", (ak.num(tight_b) >= 1))
    # selection.add("jet_dr", ((top_two_dr < 4) & (top_two_dr > 2)).to_numpy())
    selection.add("b_dr", (med_dr > 1))
    return selection

In [19]:
events = createObjects(events)
mask = createSelection(events).all("highptjet", "jets", "0Lep", "2bjet", "1tightbjet")
events = events[mask]

In [20]:
addNNScores(events)

{'features': <Array [[0, 448, 0.138, ..., 0.384, 2.03], ...] type='21803 * 13 * float64'>}


/usr/local/lib/python3.10/site-packages/coffea/ml_tools/helper.py:175: UserWarning: No format checks were performed on input!
  warnings.warn("No format checks were performed on input!")


RuntimeError: PytorchStreamReader failed locating file constants.pkl: file not found